In [1]:
import pandas as pd
import numpy as np
import h5py

In [ ]:
df = pd.read_csv('rocket_full_trainset.csv', header=None).values

In [ ]:
with h5py.File('../../Data/human-activity/train.h5', 'r') as f:
  y = f['y'][()]

target = pd.DataFrame(y, columns=['target'])
target  = target["target"]

In [7]:
X_train = df
y_train = target

In [ ]:
X_test = pd.read_csv("kaggle_test_1000.csv", header=None).values

## Modelo LR

In [11]:
from models.lr import *

In [23]:
def map_labels(y):
    unique_labels = np.unique(y)
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    return np.vectorize(label_map.get)(y), label_map

def reverse_map_labels(y_mapped, label_map):
    reverse_label_map = {idx: label for label, idx in label_map.items()}
    return np.vectorize(reverse_label_map.get)(y_mapped)

In [12]:
# Map labels to a contiguous range starting from 0
Y_train, train_label_map = map_labels(y_train)

# Verify label range
num_classes = len(np.unique(Y_train))
if np.any(Y_train >= num_classes) or np.any(Y_train < 0):
    raise ValueError("Labels in Y_train are out of bounds for the number of classes")


# Train the model
model = SoftmaxRegression()
epochs = 2000
alpha = 0.01
loss_values, weights = model.training(X_train, Y_train, epochs, alpha)

# Predictions
test_predictions = model.predict(X_test, weights)

# Evaluate the model
print("Training complete")
print("Final training loss:", loss_values[-1])

Training complete
Final training loss: 0.27243661019501497


In [24]:
# 
Y_train_original = reverse_map_labels(test_predictions, train_label_map)
print(Y_train_original.shape)
print(np.unique(Y_train_original))

(2947,)
[1. 2. 3. 4. 5. 6.]


In [22]:
df2 = pd.DataFrame({
    'ID': np.arange(1, len(Y_train_original) + 1),
    'Value': Y_train_original
})

# Guardar el DataFrame en un archivo CSV
df2.to_csv('kaggle_lr.csv', index=False)

Array guardado en output.csv con el formato especificado.
